VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os



# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = "../city_data.csv"
city_data_df = pd.read_csv(city_data)
city_data_df


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Ancud,CL,-41.8697,-73.8203,51.01,100,90,3.44,1618445435
1,Yangi Marg`ilon,UZ,40.4272,71.7189,53.60,76,100,4.00,1618445435
2,Arraial do Cabo,BR,-22.9661,-42.0278,69.80,83,40,2.30,1618445436
3,Hilo,US,19.7297,-155.0900,80.60,57,20,12.66,1618445782
4,Cheyenne,US,41.1400,-104.8203,26.60,93,90,13.80,1618445637
...,...,...,...,...,...,...,...,...,...
543,Kodīnar,IN,20.7903,70.7031,77.81,50,3,4.79,1618445624
544,Prince Rupert,CA,54.3161,-130.3201,51.80,62,1,5.75,1618445477
545,Malé,MV,4.1748,73.5089,81.97,75,41,8.86,1618445625
546,Katiola,CI,8.1333,-5.1000,78.80,83,20,2.30,1618445625


In [3]:
city_data_df.dtypes

City           object
Country        object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# Lat / Long as Locations 
locations = city_data_df[["Lat", "Lng"]]

In [6]:
# Humidity as Weight for Heatmap
city_humidity = city_data_df["Humidity"]

# Heatmap layer added to map

heatmap_fig = gmaps.figure(center=(25.0, 0), zoom_level=2)

max_intensity = np.max(city_humidity)


heat_layer = gmaps.heatmap_layer(locations, weights=city_humidity, max_intensity=100, dissipating=False, point_radius = 3)
heatmap_fig.add_layer(heat_layer)

heatmap_fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#converting unix time to date & time
# city_data_df['Date'] = pd.to_datetime(city_data_df['Date'],unit='s')
# city_data_df['Date'] = pd.to_datetime(city_data_df['Date']).dt.date
# city_data_df

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Ideal weather conditions:
    # Max Temp <95, >60
    # Wind speed <20
    # Cloudiness% <10

# city_data_df

weather_df = city_data_df[["City", "Country", "Lat","Lng","Max Temp","Wind Speed","Cloudiness"]]
weather_df.head()
weather_df.count()

City          548
Country       545
Lat           548
Lng           548
Max Temp      548
Wind Speed    548
Cloudiness    548
dtype: int64

In [9]:
idealweather_df = weather_df.loc[(weather_df["Max Temp"]<95) & (weather_df["Max Temp"]>60) & (weather_df["Wind Speed"]<20) & (weather_df["Cloudiness"]<10)]

idealweather_df.head()
idealweather_df.count()

City          68
Country       65
Lat           68
Lng           68
Max Temp      68
Wind Speed    68
Cloudiness    68
dtype: int64

In [10]:
clean_weather_df = idealweather_df.dropna()
clean_weather_df.head()
clean_weather_df.count()

City          65
Country       65
Lat           65
Lng           65
Max Temp      65
Wind Speed    65
Cloudiness    65
dtype: int64

In [11]:
clean_weather_df.head()

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness
11,Saint George,US,37.1041,-113.5841,72.00,11.50,1
12,Rikitea,PF,-23.1203,-134.9692,78.39,4.81,6
35,Bilma,NE,18.6853,12.9164,83.01,10.09,0
36,Niterói,BR,-22.8833,-43.1036,75.20,5.75,0
41,Hilton Head,US,32.2163,-80.7526,71.01,11.50,1


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = clean_weather_df.copy()
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness
11,Saint George,US,37.1041,-113.5841,72.00,11.50,1
12,Rikitea,PF,-23.1203,-134.9692,78.39,4.81,6
35,Bilma,NE,18.6853,12.9164,83.01,10.09,0
36,Niterói,BR,-22.8833,-43.1036,75.20,5.75,0
41,Hilton Head,US,32.2163,-80.7526,71.01,11.50,1
...,...,...,...,...,...,...,...
519,Makakilo City,US,21.3469,-158.0858,82.40,12.66,1
520,Doha,QA,25.2867,51.5333,80.60,10.36,0
521,Safford,US,32.8340,-109.7076,82.40,14.97,1
532,Vila Velha,BR,-20.3297,-40.2925,73.40,10.36,0


In [13]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
11,Saint George,US,37.1041,-113.5841,72.00,11.50,1,
12,Rikitea,PF,-23.1203,-134.9692,78.39,4.81,6,
35,Bilma,NE,18.6853,12.9164,83.01,10.09,0,
36,Niterói,BR,-22.8833,-43.1036,75.20,5.75,0,
41,Hilton Head,US,32.2163,-80.7526,71.01,11.50,1,
...,...,...,...,...,...,...,...,...
519,Makakilo City,US,21.3469,-158.0858,82.40,12.66,1,
520,Doha,QA,25.2867,51.5333,80.60,10.36,0,
521,Safford,US,32.8340,-109.7076,82.40,14.97,1,
532,Vila Velha,BR,-20.3297,-40.2925,73.40,10.36,0,


In [14]:
hotel_df = hotel_df.reset_index()
hotel_df

,index,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
0,11,Saint George,US,37.1041,-113.5841,72.00,11.50,1,
1,12,Rikitea,PF,-23.1203,-134.9692,78.39,4.81,6,
2,35,Bilma,NE,18.6853,12.9164,83.01,10.09,0,
3,36,Niterói,BR,-22.8833,-43.1036,75.20,5.75,0,
4,41,Hilton Head,US,32.2163,-80.7526,71.01,11.50,1,
...,...,...,...,...,...,...,...,...,...
60,519,Makakilo City,US,21.3469,-158.0858,82.40,12.66,1,
61,520,Doha,QA,25.2867,51.5333,80.60,10.36,0,
62,521,Safford,US,32.8340,-109.7076,82.40,14.97,1,
63,532,Vila Velha,BR,-20.3297,-40.2925,73.40,10.36,0,


In [15]:
hotel_df = hotel_df.drop(['index'], axis=1)
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
0,Saint George,US,37.1041,-113.5841,72.00,11.50,1,
1,Rikitea,PF,-23.1203,-134.9692,78.39,4.81,6,
2,Bilma,NE,18.6853,12.9164,83.01,10.09,0,
3,Niterói,BR,-22.8833,-43.1036,75.20,5.75,0,
4,Hilton Head,US,32.2163,-80.7526,71.01,11.50,1,
...,...,...,...,...,...,...,...,...
60,Makakilo City,US,21.3469,-158.0858,82.40,12.66,1,
61,Doha,QA,25.2867,51.5333,80.60,10.36,0,
62,Safford,US,32.8340,-109.7076,82.40,14.97,1,
63,Vila Velha,BR,-20.3297,-40.2925,73.40,10.36,0,


In [16]:
# lat = hotel_df["Lat"]

In [17]:
# lng = hotel_df["Lng"]

In [18]:
# lat_long_df = pd.DataFrame({"Lat": lat, "Lng": lng})
# lat_long_df

In [19]:
# Nearby Search Example:
    # https://maps.googleapis.com/maps/api/place/nearbysearch/json
    # https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&key=YOUR_API_KEY

# "radius" = 5000
# "type" keyword = lodging
# api key = g_key



hotel_list = []


print("--- Beginning Data Retrieval ---")

for index, row in hotel_df.iterrows():    
   
    radius = 5000
    ttype = "lodging"
    #keyword = "hotel"

    lat = row["Lat"]
    lng = row["Lng"]
    
    location = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    params = {"radius": radius, "location": location, "type": ttype, "key": g_key}
    
    
    try:
        response = requests.get(base_url, params=params)
        hotel_name = response.json()
        hotel_list.append(hotel_name["results"][0]["name"])
        print(hotel_name["results"][0]["name"]) 
    
    except:
        hotel_list.append("Nan")
        print("None found. Skipping ...")
        pass

print("--- Data Retrieval Complete ---")

--- Beginning Data Retrieval ---
Best Western Plus Abbey Inn
Pension Maro'i
Expeditions Ténére voyages
Guesthouse
Disney’s Hilton Head Island Resort
Broadwater Mariner Resort
Hotel Tesoro Los Cabos
Gran Hotel Mar del Plata
Hotels in Lakshadweep Islands
Nuba Nile
None found. Skipping ...
Ramada Hotel & Suites by Wyndham Ballina Byron
The Irish Village
University Plaza Waterfront Hotel
Islander Beach Resort
Hotel Morón
Mantra Nelson Bay
None found. Skipping ...
Hotel Las Dunas
Southern Sun Waterfront Cape Town
Regal Inn
The Ritz-Carlton, Half Moon Bay
Holiday Inn Express & Suites Pullman
مقبرة ال مجمل
Marine Club Beach Resort
Butterfly Lodge
Cattleman's Rest Motor Inn
Tuiuiu Pantanal Hotel
Nile Palace Hotel
Bushido Lodge
Hospitality Carnarvon
Takalvan Motel
Hotel Náutico de Paita
ibis Styles Mt Isa Verona
Motel 6 Sinton, TX
Abaco Beach Resort
Hampton Inn Goldsboro
None found. Skipping ...
Arco Iris
Lords Inn Somnath
Valle Aridane
Holiday Inn Express & Suites Elk Grove Central - Hwy 99
Sa

In [20]:
len(hotel_list)


65

In [21]:
# hotel_list

In [22]:
hotel_list_df = pd.DataFrame(hotel_list)
hotel_list_df

,0
0,Best Western Plus Abbey Inn
1,Pension Maro'i
2,Expeditions Ténére voyages
3,Guesthouse
4,Disney’s Hilton Head Island Resort
...,...
60,Marriott's Ko Olina Beach Club
61,Movenpick Hotel Doha
62,SpringHill Suites by Marriott Thatcher
63,Quality Suites Vila Velha


In [23]:
hotel_df["Hotel Name"] = hotel_list
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
0,Saint George,US,37.1041,-113.5841,72.00,11.50,1,Best Western Plus Abbey Inn
1,Rikitea,PF,-23.1203,-134.9692,78.39,4.81,6,Pension Maro'i
2,Bilma,NE,18.6853,12.9164,83.01,10.09,0,Expeditions Ténére voyages
3,Niterói,BR,-22.8833,-43.1036,75.20,5.75,0,Guesthouse
4,Hilton Head,US,32.2163,-80.7526,71.01,11.50,1,Disney’s Hilton Head Island Resort
...,...,...,...,...,...,...,...,...
60,Makakilo City,US,21.3469,-158.0858,82.40,12.66,1,Marriott's Ko Olina Beach Club
61,Doha,QA,25.2867,51.5333,80.60,10.36,0,Movenpick Hotel Doha
62,Safford,US,32.8340,-109.7076,82.40,14.97,1,SpringHill Suites by Marriott Thatcher
63,Vila Velha,BR,-20.3297,-40.2925,73.40,10.36,0,Quality Suites Vila Velha


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map

max_temp = hotel_df["Max Temp"]

heatmap_fig = gmaps.figure(center=(25.0, 0), zoom_level=2)

max_intensity = np.max(city_humidity)


heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, max_intensity=100, dissipating=False, point_radius = 3)

heat_marker = gmaps.marker_layer(locations, info_box_content = hotel_info)

heatmap_fig.add_layer(heat_marker)

heatmap_fig.add_layer(heat_layer)

heatmap_fig


# Display figure


Figure(layout=FigureLayout(height='420px'))